# Combined Memory

In [ ]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [ ]:
pip show langchain

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.memory import (ConversationBufferMemory, 
                              ConversationSummaryMemory, 
                              CombinedMemory)
from langchain.globals import set_verbose

set_verbose(True)

In [ ]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 100)

In [ ]:
TEMPLATE = '''
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Past messages:
{message_buffer_log}

Conversation summary:
{message_summary_log}

Human: {question}
AI:
'''

prompt_template = PromptTemplate.from_template(template = TEMPLATE)

In [ ]:
chat_buffer_memory = ConversationBufferMemory(memory_key = 'message_buffer_log', 
                                              input_key = 'question',
                                              return_messages = False)

chat_summary_memory = ConversationSummaryMemory(llm = ChatOpenAI(), 
                                                memory_key = 'message_summary_log', 
                                                input_key = 'question',
                                                return_messages = False)

chat_combined_memory = CombinedMemory(memories = [chat_buffer_memory, 
                                                  chat_summary_memory])

In [ ]:
print(chat_combined_memory.load_memory_variables({})['message_summary_log'])

In [ ]:
chain = LLMChain(llm = chat, 
                 prompt = prompt_template, 
                 memory = chat_combined_memory)

In [ ]:
chain.invoke({'question':'''Can you give me an interesting fact I probably didn't know about?'''})
# chain.invoke({'question':'''Can you elaborate a bit more on this fact?'''})